In [1]:
import pandas as pd
import numpy as np
import scipy
from scipy import stats
from numpy import sqrt
from statsmodels.stats.multitest import multipletests
from scipy.stats import mannwhitneyu
from numpy import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [2]:
answer=pd.read_excel(r'C:\Users\Client\OneDrive - Queen Mary, University of London\ANSWER ALS\ANSwer res\proteomics\4_matrix\AnswerALS.xlsx', header=0, index_col= "Gene names")
answer.head()

,ALS,ALS.1,ALS.2,ALS.3,ALS.4,ALS.5,ALS.6,ALS.7,ALS.8,ALS.9,...,Control.10,Control.11,Control.12,Control.13,Control.14,Control.15,Control.16,Control.17,Control.18,Control.19
Gene names,,,,,,,,,,,,,,,,,,,,,
UBA6,2626.16,2588.04,2507.03,2913.20,2865.30,3357.25,2967.33,3116.56,3013.63,2639.38,...,2047.42,2662.27,3188.02,2522.14,2606.57,2443.07,3209.72,3040.75,2925.38,2915.48
ESYT2,3985.19,3860.51,3931.82,3926.29,4940.13,4587.97,3472.10,4155.66,3633.70,5932.34,...,4071.36,5026.31,5002.96,4016.65,4359.47,3860.47,5079.03,3744.31,5262.26,3803.53
SHOT1,17934.00,16037.20,18116.60,15280.10,19765.40,14493.70,15945.10,15707.10,17674.20,23334.40,...,17266.60,17548.30,15819.40,15412.40,14926.00,16555.90,15631.90,16093.40,15879.00,15294.80
ILVBL,15656.00,19963.40,14834.90,15015.00,15356.70,18380.80,14549.00,14402.90,17603.90,21581.90,...,15379.40,16888.10,16864.60,15095.60,16127.10,17748.70,16544.00,16160.80,16737.70,16794.30
SPD2B,5242.79,5763.16,5432.44,4587.41,5445.52,5633.45,5827.67,5705.44,5218.67,4173.28,...,5780.46,5016.27,4814.92,5955.49,2665.17,5396.64,4170.71,5577.04,5083.72,5189.22


In [3]:
#Replace missing values (0) with NA
answer.replace (0, np.nan, inplace=True)

In [4]:
answer.shape

(3264, 365)

In [5]:
# Columns belonging to Group 1 and 2
ALS_columns = [col for col in answer.columns if "ALS" in col]
Control_columns = [col for col in answer.columns if "Control" in col]


# Filter rows where at least 50% of Group 1 values are present, do the same for Group 2
filtered_als = answer[answer[ALS_columns].notna().mean(axis=1) >= 0.5]
cont_filtered = filtered_als[filtered_als[Control_columns].notna().mean(axis=1) >= 0.5]

In [6]:
cont_filtered.shape

(3264, 365)

In [7]:
cont_filtered.max().max()

53497500000.0

In [8]:
cont_filtered.min().min()

13.5798

In [9]:
# Set the new range
min_range = 1
max_range = 100

In [10]:
cont_filtered_min = cont_filtered.min().min()
cont_filtered_max = cont_filtered.max().max()
scaled_cont_filtered = (cont_filtered - cont_filtered_min) / (cont_filtered_max - cont_filtered_min)
scaled_data = (scaled_cont_filtered * (max_range - min_range)) + min_range
scaled_data.head()

,ALS,ALS.1,ALS.2,ALS.3,ALS.4,ALS.5,ALS.6,ALS.7,ALS.8,ALS.9,...,Control.10,Control.11,Control.12,Control.13,Control.14,Control.15,Control.16,Control.17,Control.18,Control.19
Gene names,,,,,,,,,,,,,,,,,,,,,
UBA6,1.000005,1.000005,1.000005,1.000005,1.000005,1.000006,1.000005,1.000006,1.000006,1.000005,...,1.000004,1.000005,1.000006,1.000005,1.000005,1.000004,1.000006,1.000006,1.000005,1.000005
ESYT2,1.000007,1.000007,1.000007,1.000007,1.000009,1.000008,1.000006,1.000008,1.000007,1.000011,...,1.000008,1.000009,1.000009,1.000007,1.000008,1.000007,1.000009,1.000007,1.000010,1.000007
SHOT1,1.000033,1.000030,1.000034,1.000028,1.000037,1.000027,1.000029,1.000029,1.000033,1.000043,...,1.000032,1.000032,1.000029,1.000028,1.000028,1.000031,1.000029,1.000030,1.000029,1.000028
ILVBL,1.000029,1.000037,1.000027,1.000028,1.000028,1.000034,1.000027,1.000027,1.000033,1.000040,...,1.000028,1.000031,1.000031,1.000028,1.000030,1.000033,1.000031,1.000030,1.000031,1.000031
SPD2B,1.000010,1.000011,1.000010,1.000008,1.000010,1.000010,1.000011,1.000011,1.000010,1.000008,...,1.000011,1.000009,1.000009,1.000011,1.000005,1.000010,1.000008,1.000010,1.000009,1.000010


In [11]:
group1= scaled_data.loc[:, scaled_data.columns.str.startswith('Control.')]
group2= scaled_data.loc[:, scaled_data.columns.str.startswith('ALS.')]

In [12]:
# Perform Mann-Whitney U test row-wise, drop NA values 
stat, p_value = mannwhitneyu(group1.iloc[0], group2.iloc[0], alternative='two-sided', method='auto')

stats = []
p_vals = []
for i in range(group1.shape[0]):
    stat, p_value = mannwhitneyu(group1.iloc[i].dropna(), group2.iloc[i].dropna(), alternative='two-sided', method='auto')
    stats.append(stat)
    p_vals.append(p_value)

In [13]:
scaled_data['U-stat'] = stats
scaled_data['P-value'] = p_vals
scaled_data.head()

,ALS,ALS.1,ALS.2,ALS.3,ALS.4,ALS.5,ALS.6,ALS.7,ALS.8,ALS.9,...,Control.12,Control.13,Control.14,Control.15,Control.16,Control.17,Control.18,Control.19,U-stat,P-value
Gene names,,,,,,,,,,,,,,,,,,,,,
UBA6,1.000005,1.000005,1.000005,1.000005,1.000005,1.000006,1.000005,1.000006,1.000006,1.000005,...,1.000006,1.000005,1.000005,1.000004,1.000006,1.000006,1.000005,1.000005,2281.0,0.027776
ESYT2,1.000007,1.000007,1.000007,1.000007,1.000009,1.000008,1.000006,1.000008,1.000007,1.000011,...,1.000009,1.000007,1.000008,1.000007,1.000009,1.000007,1.000010,1.000007,2915.0,0.428554
SHOT1,1.000033,1.000030,1.000034,1.000028,1.000037,1.000027,1.000029,1.000029,1.000033,1.000043,...,1.000029,1.000028,1.000028,1.000031,1.000029,1.000030,1.000029,1.000028,2193.0,0.015814
ILVBL,1.000029,1.000037,1.000027,1.000028,1.000028,1.000034,1.000027,1.000027,1.000033,1.000040,...,1.000031,1.000028,1.000030,1.000033,1.000031,1.000030,1.000031,1.000031,1959.0,0.003296
SPD2B,1.000010,1.000011,1.000010,1.000008,1.000010,1.000010,1.000011,1.000011,1.000010,1.000008,...,1.000009,1.000011,1.000005,1.000010,1.000008,1.000010,1.000009,1.000010,2631.0,0.152862


In [14]:
# Apply Benjamini-Hochberg correction (optional)
_, p_adjusted, _, _ = multipletests(scaled_data['P-value'], method='fdr_bh')
scaled_data['p_adjusted'] = p_adjusted
scaled_data.head()

,ALS,ALS.1,ALS.2,ALS.3,ALS.4,ALS.5,ALS.6,ALS.7,ALS.8,ALS.9,...,Control.13,Control.14,Control.15,Control.16,Control.17,Control.18,Control.19,U-stat,P-value,p_adjusted
Gene names,,,,,,,,,,,,,,,,,,,,,
UBA6,1.000005,1.000005,1.000005,1.000005,1.000005,1.000006,1.000005,1.000006,1.000006,1.000005,...,1.000005,1.000005,1.000004,1.000006,1.000006,1.000005,1.000005,2281.0,0.027776,0.365294
ESYT2,1.000007,1.000007,1.000007,1.000007,1.000009,1.000008,1.000006,1.000008,1.000007,1.000011,...,1.000007,1.000008,1.000007,1.000009,1.000007,1.000010,1.000007,2915.0,0.428554,0.713310
SHOT1,1.000033,1.000030,1.000034,1.000028,1.000037,1.000027,1.000029,1.000029,1.000033,1.000043,...,1.000028,1.000028,1.000031,1.000029,1.000030,1.000029,1.000028,2193.0,0.015814,0.330605
ILVBL,1.000029,1.000037,1.000027,1.000028,1.000028,1.000034,1.000027,1.000027,1.000033,1.000040,...,1.000028,1.000030,1.000033,1.000031,1.000030,1.000031,1.000031,1959.0,0.003296,0.292441
SPD2B,1.000010,1.000011,1.000010,1.000008,1.000010,1.000010,1.000011,1.000011,1.000010,1.000008,...,1.000011,1.000005,1.000010,1.000008,1.000010,1.000009,1.000010,2631.0,0.152862,0.510688


In [15]:
_, p_adjusted, _, _ = multipletests(scaled_data['P-value'], method='bonferroni')
scaled_data['p_adjusted_bonf'] = p_adjusted
scaled_data.head()

,ALS,ALS.1,ALS.2,ALS.3,ALS.4,ALS.5,ALS.6,ALS.7,ALS.8,ALS.9,...,Control.14,Control.15,Control.16,Control.17,Control.18,Control.19,U-stat,P-value,p_adjusted,p_adjusted_bonf
Gene names,,,,,,,,,,,,,,,,,,,,,
UBA6,1.000005,1.000005,1.000005,1.000005,1.000005,1.000006,1.000005,1.000006,1.000006,1.000005,...,1.000005,1.000004,1.000006,1.000006,1.000005,1.000005,2281.0,0.027776,0.365294,1.0
ESYT2,1.000007,1.000007,1.000007,1.000007,1.000009,1.000008,1.000006,1.000008,1.000007,1.000011,...,1.000008,1.000007,1.000009,1.000007,1.000010,1.000007,2915.0,0.428554,0.713310,1.0
SHOT1,1.000033,1.000030,1.000034,1.000028,1.000037,1.000027,1.000029,1.000029,1.000033,1.000043,...,1.000028,1.000031,1.000029,1.000030,1.000029,1.000028,2193.0,0.015814,0.330605,1.0
ILVBL,1.000029,1.000037,1.000027,1.000028,1.000028,1.000034,1.000027,1.000027,1.000033,1.000040,...,1.000030,1.000033,1.000031,1.000030,1.000031,1.000031,1959.0,0.003296,0.292441,1.0
SPD2B,1.000010,1.000011,1.000010,1.000008,1.000010,1.000010,1.000011,1.000011,1.000010,1.000008,...,1.000005,1.000010,1.000008,1.000010,1.000009,1.000010,2631.0,0.152862,0.510688,1.0


In [16]:
#df['average'] = df.mean(numeric_only=True, axis=1)
scaled_data['Mean Control'] = scaled_data.loc[:, scaled_data.columns.str.startswith('Control.')].mean(axis=1)
scaled_data['Mean ALS'] = scaled_data.loc[:, scaled_data.columns.str.startswith('ALS.')].mean(axis=1)

In [17]:
#Fold Change
scaled_data["FC ALS-C"]= scaled_data["Mean ALS"]/scaled_data["Mean Control"]

In [18]:
#Log 2 FC
scaled_data["Log2FC ALS-C"]=np.log2(scaled_data["FC ALS-C"])

In [19]:
scaled_data.to_excel(r'C:\Users\Client\OneDrive - Queen Mary, University of London\ANSWER ALS\ANSwer res\proteomics\4_matrix\AnswerALS scaled Mann-W.xlsx')

In [20]:
# Sort the DataFrame by the absolute values of the column
net_sorted = scaled_data.loc[scaled_data["Log2FC ALS-C"].abs().sort_values(ascending=False).index]


In [21]:
net_sorted.to_excel(r'C:\Users\Client\OneDrive - Queen Mary, University of London\ANSWER ALS\ANSwer res\proteomics\4_matrix\AnswerALS scaled Mann-W filtered.xlsx')

In [22]:
# Filter rows where 'P-value' <= 0.05
filtered_net = net_sorted[net_sorted['P-value'] <= 0.05]

In [23]:
filtered_net.to_excel(r'C:\Users\Client\OneDrive - Queen Mary, University of London\ANSWER ALS\ANSwer res\proteomics\4_matrix\AnswerALS scaled Mann-W sorted.xlsx')